# Kiva Project: Computer Science

Foundation of Computer Science Project on Kiva Dataset

Mattia Rigiroli, 819987

21-01-2020

Importo tutte le librerie necessarie:

In [1]:
import pandas as pd
import numpy as np
import time
import datetime
import random

In [2]:
# Impostiamo il numero di cifre decimali da mostrare
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# 1- Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.

In [3]:
loan_lenders=pd.read_csv(r"C:\Users\matti\OneDrive - Universita degli Studi di Milano-Bicocca\Uni\LM-DataScience\1-Anno\Computer Science\Final Project\Final Project CS\Kiva_Dataset\loans_lenders.csv")

In [4]:
loan_lenders.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


In [5]:
loan_lenders.shape

(1387432, 2)

In [6]:
loan_short=loan_lenders[0:100]
df = pd.DataFrame({'loan_id':[], 'lender':[]})
for i in range(0,len(loan_short)):
    loan_id=loan_short["loan_id"].iloc[i]
    lenders=loan_short['lenders'].iloc[i]
    lenders_list=lenders.split(',')
    for k in range(0,len(lenders_list)):
        # Aggiungo al DataFrame df la coppia id:lender
        df = pd.DataFrame(np.array([[loan_id,lenders_list[k]]]),columns=["loan_id", "lender"]).append(df, ignore_index=True)
df.head()

,loan_id,lender
0,338792,emily3379
1,338792,shannon3680
2,338792,anders5148
3,338792,scott9318
4,282917,rs7257


Questo metodo applicato all'intero dataset risulta molto lento e inefficace. Ho quindi applicato un altra metodologia.

Elaborazione sull'intero dataset:

In [7]:
# Creo una lista di ID 
id_list=[x for x in loan_lenders['loan_id']]
id_list[0:10]

[483693,
 483738,
 485000,
 486087,
 534428,
 558112,
 563395,
 575414,
 578029,
 551251]

In [8]:
# Creo una lista di lenders
lenders_list=[y for x in loan_lenders['lenders'] for y in x.split(',')]
lenders_list[0:20]

['muc888',
 ' sam4326',
 ' camaran3922',
 ' lachheb1865',
 ' rebecca3499',
 ' karlheinz4543',
 ' jerrydb',
 ' paula8951',
 ' gmct',
 ' amra9383',
 ' r3922',
 ' brian9451',
 ' shree8053',
 ' alan5513',
 ' oisin3389',
 ' helle8622',
 ' bo3186',
 ' ric8947',
 ' daniel98469874',
 ' nick9464']

In [9]:
len(lenders_list)

28293931

In [10]:
# Creo una lista che contiene il numero di lenders per ciascun ID
# In questo caso, al primo loan_id corrispondono 40 lenders e così via. 
lunghezza=[len(x.split(',')) for x in loan_lenders['lenders']]
lunghezza[0:10]

[40, 15, 15, 13, 19, 13, 11, 23, 13, 32]

In [11]:
# Aggiungo ad una nuova lista ciascun ID presente nella lista id_list per il numero di volte 
# presente in lunghezza (n di lenders per ciascun ID). 
id_list1=[]
for i in range(0,len(lunghezza)):
    id_list1.append([id_list[i]]*lunghezza[i])
# Il risultato è una lista di liste: estraggo perciò i singoli ID e li metto in un'unica lista. 
id_finale=[y for x in id_list1 for y in x]
id_finale[0:10]    

[483693,
 483693,
 483693,
 483693,
 483693,
 483693,
 483693,
 483693,
 483693,
 483693]

In [12]:
# Verifico che il numero di ID sia uguale al numero di lenders. 
len(id_finale)

28293931

In [13]:
# Creo il dataset loan lender normalizzato
loan_lender_norm = pd.DataFrame({'loan_id':id_finale, 'lender':lenders_list})

In [14]:
loan_lender_norm

,loan_id,lender
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175


# 2- For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

In [15]:
loans=pd.read_csv(r"C:\Users\matti\OneDrive - Universita degli Studi di Milano-Bicocca\Uni\LM-DataScience\1-Anno\Computer Science\Final Project\Final Project CS\Kiva_Dataset\loans.csv")

In [16]:
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.00,125.00,funded,General Store,Retail,...,2014-01-15 04:48:22.000 +0000,7.00,3,2,1,NaN,female,true,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.00,400.00,funded,Used Clothing,Clothing,...,2014-02-25 06:42:06.000 +0000,8.00,11,2,1,NaN,female,true,monthly,field_partner
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.00,400.00,funded,General Store,Retail,...,2014-01-24 23:06:18.000 +0000,14.00,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.00,625.00,funded,Farming,Agriculture,...,2014-01-22 05:29:28.000 +0000,14.00,21,2,1,user_favorite,female,true,monthly,field_partner
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.00,425.00,funded,Farming,Agriculture,...,2014-01-14 17:29:27.000 +0000,7.00,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner


In [17]:
type(loans['disburse_time'][1])

str

In [18]:
type(loans['planned_expiration_time'][1])

str

In [19]:
loans['planned_expiration_time'] = pd.to_datetime(loans['planned_expiration_time'])
loans['disburse_time']= pd.to_datetime(loans['disburse_time'])

In [20]:
loans['loans_duration']=abs(loans['planned_expiration_time']-loans['disburse_time'])

In [21]:
loans['loans_duration'][0:5]

0   53 days 19:30:06
1   96 days 14:25:07
2   37 days 13:10:05
3   34 days 19:10:02
4   57 days 22:10:02
Name: loans_duration, dtype: timedelta64[ns]

Controllo sui valori mancanti: quando disburse_time o planned_expiration_time sono nulle (o entrambe sono nulle), loans_duration è nulla.

In [22]:
# Controllo la persenza di valori nulli nelle due variabili considerate:
print(loans['disburse_time'].isnull().any())
print(loans['planned_expiration_time'].isnull().any())
print(loans['loans_duration'].isnull().any())

True
True
True


In [23]:
# Valore nullo in planned_expiration_time e loans_duration
loans[['disburse_time','planned_expiration_time','loans_duration']].iloc[64]

disburse_time              2010-11-18 08:00:00+00:00
planned_expiration_time                          NaT
loans_duration                                   NaT
Name: 64, dtype: object

In [24]:
# Valore nullo in disburse_time e loans_duration
loans[['disburse_time','planned_expiration_time','loans_duration']].iloc[31106]

disburse_time                                    NaT
planned_expiration_time    2017-12-06 16:41:36+00:00
loans_duration                                   NaT
Name: 31106, dtype: object

In [25]:
loans[['disburse_time','planned_expiration_time','loans_duration']].isna().sum()  

disburse_time                2813
planned_expiration_time    371834
loans_duration             374645
dtype: int64

In [26]:
loans[['loan_id','loans_duration']].head(10)

,loan_id,loans_duration
0,657307,53 days 19:30:06
1,657259,96 days 14:25:07
2,658010,37 days 13:10:05
3,659347,34 days 19:10:02
4,656933,57 days 22:10:02
5,659605,35 days 18:30:07
6,660240,31 days 23:50:11
7,661601,46 days 01:50:03
8,661960,33 days 16:10:03
9,662117,68 days 05:40:02


In [27]:
loans[['loan_id','loans_duration']].tail(10)

,loan_id,loans_duration
1419597,984735,63 days 06:20:02
1419598,985764,46 days 02:40:05
1419599,986756,44 days 12:20:04
1419600,986789,40 days 11:50:04
1419601,987606,42 days 08:30:11
1419602,988180,39 days 17:00:03
1419603,988213,39 days 08:40:07
1419604,989109,51 days 14:20:04
1419605,989143,63 days 00:50:02
1419606,989240,61 days 12:50:06


# 3- Find the lenders that have funded at least twice.

In [28]:
loan_lender_norm.head()

,loan_id,lender
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499


In [29]:
d_funded=loan_lender_norm.groupby('lender').count()
d_funded.rename(columns={'loan_id': 'n loan x lender'}, inplace=True)

In [30]:
d_funded.head()

,n loan x lender
lender,
000,39
00000,39
0002,70
00mike00,1
0101craign0101,71


In [31]:
# Seleziono solamente i lenders che hanno finanziato almeno due volte e creo un nuovo DataFrame. 
d_funded_2=d_funded[d_funded['n loan x lender']>=2].sort_values(by='n loan x lender', ascending=False)
d_funded_2

,n loan x lender
lender,
gooddogg1,151307
trolltech4460,148347
gmct,127089
nms,100360
themissionbeltco,76986
...,...
omurhu9493,2
beth36573246,2
omri1441,2


# 4-  For each country, compute how many loans have involved that country as borrowers.

In [32]:
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,loans_duration
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.00,125.00,funded,General Store,Retail,...,7.00,3,2,1,NaN,female,true,irregular,field_partner,53 days 19:30:06
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.00,400.00,funded,Used Clothing,Clothing,...,8.00,11,2,1,NaN,female,true,monthly,field_partner,96 days 14:25:07
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.00,400.00,funded,General Store,Retail,...,14.00,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,37 days 13:10:05
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.00,625.00,funded,Farming,Agriculture,...,14.00,21,2,1,user_favorite,female,true,monthly,field_partner,34 days 19:10:02
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.00,425.00,funded,Farming,Agriculture,...,7.00,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,57 days 22:10:02


In [33]:
# Il dataset loans si riferisce ai mutui perciò ho raggruppato per paese
# contando quante volte ciauscun paese compariva.
count_borrowers=pd.DataFrame(loans.groupby('country_name').count()['loan_id'])
count_borrowers.rename(columns={'loan_id': 'tot_loan'}, inplace=True)
count_borrowers=count_borrowers.sort_values(by='tot_loan', ascending=False)
count_borrowers

,tot_loan
country_name,
Philippines,285336
Kenya,143699
Peru,86000
Cambodia,79701
El Salvador,64037
...,...
Canada,1
Papua New Guinea,1
Botswana,1


# 5- For each country, compute the overall amount of money borrowed.

In [34]:
cash_borrowed=pd.DataFrame(loans.groupby('country_name').sum()['loan_amount'])
cash_borrowed.rename(columns={'loan_amount': 'overall_amount'}, inplace=True)
cash_borrowed=cash_borrowed.sort_values(by='overall_amount', ascending=False)
cash_borrowed

,overall_amount
country_name,
Philippines,97984600.00
Peru,79437775.00
Kenya,66735975.00
Paraguay,53964700.00
Cambodia,51613525.00
...,...
Virgin Islands,10000.00
Vanuatu,9250.00
Botswana,8000.00


Le Filippine sono lo stato che ha preso a prestito la quantità di denaro più alta. 

In [35]:
# Calcolo il totale di soldi finanziati che servirà nei punti successivi. 
totale=sum(cash_borrowed['overall_amount'])
totale

1181437300.0

# 6- Like the previous point, but expressed as a percentage of the overall amount lent.


In [36]:
perc_cash_borrowed=pd.DataFrame(((loans.groupby('country_name').sum()['loan_amount'])/totale)*100)
perc_cash_borrowed.rename(columns={'loan_amount': 'perc_total_amount'}, inplace=True)
perc_cash_borrowed=perc_cash_borrowed.sort_values(by='perc_total_amount', ascending=False)
perc_cash_borrowed

,perc_total_amount
country_name,
Philippines,8.29
Peru,6.72
Kenya,5.65
Paraguay,4.57
Cambodia,4.37
...,...
Virgin Islands,0.00
Vanuatu,0.00
Botswana,0.00


Le Filippine hanno preso a prestito l'8,3% del denaro prestato totale.

# 7- Like the three previous points, but split for each year (with respect to disburse time).


In [37]:
# Creo una variabile anno per ciascun loans:
loans['disburse_years']=pd.to_datetime(loans['disburse_time']).dt.year

In [38]:
# Calcolo il totale per ogni anno:
tot_year=pd.DataFrame(loans.groupby('disburse_years').sum()['loan_amount'])
tot_year.rename(columns={'loan_amount': 'year_overall_amount'}, inplace=True)
tot_year

,year_overall_amount
disburse_years,
2005.00,102850.00
2006.00,1376575.00
2007.00,15446525.00
2008.00,39423050.00
2009.00,59689475.00
2010.00,72609150.00
2011.00,93699300.00
2012.00,119977575.00
2013.00,132043925.00


In [39]:
sum(tot_year['year_overall_amount'])
# Il totale non corrisponde al totale calcolato al punto precedente
# ed è minore in quanto vi sono più valori nulli in
# disrburse time rispetto che al country_name. 

1170496550.0

In [40]:
loans[['disburse_time','disburse_years','country_name']].isna().sum()  

disburse_time     2813
disburse_years    2813
country_name         0
dtype: int64

In [41]:
perc_cash_borrowed_year=pd.DataFrame(((loans.groupby(['country_name','disburse_years']).sum()['loan_amount'])/(loans.groupby('disburse_years').sum()['loan_amount']))*100)
perc_cash_borrowed_year.rename(columns={'loan_amount': 'perc_total_amount'}, inplace=True)
perc_cash_borrowed_year

perc_total_amount
country_name disburse_years                   
Afghanistan  2007.00                      1.26
             2008.00                      0.93
             2009.00                      0.98
             2010.00                      0.78
             2011.00                      0.26
...                                        ...
Zimbabwe     2013.00                      0.51
             2014.00                      0.86
             2015.00                      0.46
             2016.00                      0.49
             2017.00                      0.75

[748 rows x 1 columns]

Ad esempio, l'Afghanistan nel 2007 ha preso in prestito l'1,26% del totale di soldi presi in prestito nel 2007.

# 8- For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

In [42]:
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,loans_duration,disburse_years
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.00,125.00,funded,General Store,Retail,...,3,2,1,NaN,female,true,irregular,field_partner,53 days 19:30:06,2013.00
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.00,400.00,funded,Used Clothing,Clothing,...,11,2,1,NaN,female,true,monthly,field_partner,96 days 14:25:07,2013.00
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.00,400.00,funded,General Store,Retail,...,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,37 days 13:10:05,2014.00
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.00,625.00,funded,Farming,Agriculture,...,21,2,1,user_favorite,female,true,monthly,field_partner,34 days 19:10:02,2014.00
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.00,425.00,funded,Farming,Agriculture,...,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,57 days 22:10:02,2013.00


In [43]:
# Creo un sottodataset di loans con le sole variabili d'interesse
# per snellire la procedura di merge. 
loans_sel=loans[['loan_id','num_lenders_total','funded_amount']]

In [44]:
# Unisco il dataset normnalizzato con il sottodataset di loans
df=pd.merge(loan_lender_norm,loans_sel,on='loan_id')
df.head()

,loan_id,lender,num_lenders_total,funded_amount
0,483693,muc888,44,1225.00
1,483693,sam4326,44,1225.00
2,483693,camaran3922,44,1225.00
3,483693,lachheb1865,44,1225.00
4,483693,rebecca3499,44,1225.00


In [45]:
# Calcolo il contributo di ciascun lenders per ciascun loans
df['$lenders']=df['funded_amount']/df['num_lenders_total']
df.head()

,loan_id,lender,num_lenders_total,funded_amount,$lenders
0,483693,muc888,44,1225.00,27.84
1,483693,sam4326,44,1225.00,27.84
2,483693,camaran3922,44,1225.00,27.84
3,483693,lachheb1865,44,1225.00,27.84
4,483693,rebecca3499,44,1225.00,27.84


In [46]:
df1=pd.DataFrame(df.groupby('lender').sum()['$lenders'])
#df1=df1.sort_values(by='$lenders', ascending=False)
df1=df1.reset_index()
df1

,lender,$lenders
0,000,1485.31
1,00000,1249.13
2,0002,2201.18
3,00mike00,38.46
4,0101craign0101,2424.09
...,...,...
1639021,zzanita,62.50
1639022,zzcyna7269,44.29
1639023,zzinnia,32.76
1639024,zzmcfate,1889.70


# 9- For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

In [47]:
lender=pd.read_csv(r"C:\Users\matti\OneDrive - Universita degli Studi di Milano-Bicocca\Uni\LM-DataScience\1-Anno\Computer Science\Final Project\Final Project CS\Kiva_Dataset\lenders.csv")

In [48]:
lender.head()

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.00,NaN,0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.00,NaN,0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,nan,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.00,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,nan,NaN,0


In [49]:
lender.isna().sum()

permanent_name             0
display_name            2768
city                 1619306
state                1713481
country_code         1458635
member_since               0
occupation           1844514
loan_because         2174852
loan_purchase_num     894281
invited_by           1852349
num_invited                0
dtype: int64

In [50]:
round(sum(lender['country_code'].isnull())/len(lender)*100,2)

62.09

Notiamo che la presenza di valori mancanti per country_code è rilevante:
imputo i valori mancati per country_code simulando la distribuzione dei valori di country_code non mancanti. 

In [51]:
# Creo un dataset con tutti i valori nulli per country_code:
lender_vuoto=lender[lender['country_code'].isnull()==True]
lender_vuoto.shape

(1458635, 11)

In [52]:
# Creo un dataset con tutti i valori non nulli per country_code:
lender_pieno=lender[lender['country_code'].isnull()==False]
lender_pieno.shape

(890539, 11)

In [53]:
# Conto quante volte appare ogni stato nel dataset relativo ai valori non nulli:
n_stati=pd.DataFrame(lender_pieno.groupby('country_code').size())
n_stati.rename(columns={0: 'n_stati'}, inplace=True)
n_stati=n_stati.reset_index()
n_stati.head()

,country_code,n_stati
0,AD,15
1,AE,1043
2,AF,228
3,AG,8
4,AI,4


In [54]:
# Creo una lista in cui ogni country_code è ripetuto per il numero di volte che 
# compare nel dataset dei valori non nulli. 
# Creo la lista di stati.
lista_stati=[x for x in n_stati['country_code']]
# Conto quante volte compare ogni stato
volte=[x for x in n_stati['n_stati']]
# Ripeto ogni stato per il numero di volte in cui compare: creo la lista da cui posso 
# poi estrarre in modo casuale dei valori per country_code. 
ll=[]
for i in range(0,len(n_stati)):
    ll.append([lista_stati[i]]*volte[i])
lista_stati_real=[y for x in ll for y in x]

In [55]:
# Vado a riempire la colonna country_code nel dataset vuoto simulando in modo
# casuale (basandomi sulla lista appena creata) il country_code. 
random.seed(9999)
ll_casual=[]
for i in range (0,len(lender_vuoto)):
    ll_casual.append(random.sample(lista_stati_real,1)[0])
lender_vuoto['country_code']=ll_casual

C:\Users\matti\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [56]:
len(ll_casual)

1458635

In [57]:
# Unisco i due dataset:
lender_pieno=lender_pieno.append(lender_vuoto)

In [58]:
lender_pieno.head()

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
16,naresh2074,Naresh,NaN,NaN,US,1461303119,NaN,NaN,3.00,NaN,0
31,christina27976796,Christina,NaN,NaN,US,1461303341,NaN,NaN,1.00,Peter Tan,0
37,vikas1098,Vikas,Bengaluru,NaN,IN,1461301846,Software Engineer,NaN,2.00,NaN,0
39,qian1385,Qian,NaN,NaN,US,1461302036,NaN,NaN,2.00,NaN,0
42,xigg8769,Darla,Madison,NaN,US,1461302503,NaN,NaN,nan,NaN,0


In [59]:
lender_sel=lender_pieno[['permanent_name','country_code']]
lender_sel.rename(columns={'permanent_name': 'lender'}, inplace=True)

C:\Users\matti\Anaconda3\lib\site-packages\pandas\core\frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [60]:
# Lenders e rispettivi country_code
lender_sel.head()

,lender,country_code
16,naresh2074,US
31,christina27976796,US
37,vikas1098,IN
39,qian1385,US
42,xigg8769,US


In [61]:
# Contributo totale per ciascun lender
df1.head()

,lender,$lenders
0,000,1485.31
1,00000,1249.13
2,0002,2201.18
3,00mike00,38.46
4,0101craign0101,2424.09


In [62]:
df2=pd.merge(df1,lender_sel,on='lender')
df2.head()

,lender,$lenders,country_code
0,000,26.79,US
1,0161130,25.00,US
2,0169713,25.00,US
3,0197462,25.00,IT
4,0206338,29.69,US


In [63]:
df3=pd.DataFrame(df2.groupby('country_code').sum()['$lenders'])
#df3=df3.reset_index()
df3

,$lenders
country_code,
AD,101.19
AE,101048.68
AF,6556.19
AI,29.69
AL,1190.34
...,...
YE,6462.32
YT,64.66
ZA,30814.84


Qui ho calcolato l'ammontare di denaro prestato da ogni stato.

In [64]:
cash_borrowed=pd.DataFrame(loans.groupby('country_code').sum()['loan_amount'])
cash_borrowed.rename(columns={'loan_amount': 'overall_amount'}, inplace=True)
#cash_borrowed=cash_borrowed.reset_index()
cash_borrowed

,overall_amount
country_code,
AF,1967950.00
AL,4307350.00
AM,22950475.00
AZ,14784625.00
BA,477250.00
...,...
XK,3083025.00
YE,3444000.00
ZA,1006525.00


Qui ho calcolato l'ammontare di denaro preso a prestito da ogni stato.

In [65]:
difference=pd.DataFrame(df3['$lenders']-cash_borrowed['overall_amount'])
difference.rename(columns={0: 'difference_amount'}, inplace=True)
difference=difference.reset_index()
difference.rename(columns={'index': 'country_code'}, inplace=True)
difference
# Vi sono dei valori nulli in quanto alcuni stati non hanno mai prestato / preso 
# a prestito... (e quindi hanno magari solamente prestato / preso a prestito))

,country_code,difference_amount
0,AD,nan
1,AE,nan
2,AF,-1961393.81
3,AI,nan
4,AL,-4306159.66
...,...,...
215,YE,-3437537.68
216,YT,nan
217,ZA,-975710.16
218,ZM,-1978309.01


In [66]:
# Creo due dizionari che contengono i precedenti dataset:
# chiave (stato) : valore ($)
diz_lenders=df3.to_dict()
diz_lenders=diz_lenders['$lenders']
diz_borrowers=cash_borrowed.to_dict()
diz_borrowers=diz_borrowers['overall_amount']

In [67]:
for i in range(0,len(difference)):
  #se il valore in difference è nullo, vado ad insierire il valore che si trova in uno dei due dizionari precedentemente creati.
  if pd.isna(difference.iloc[i]['difference_amount'])==True:                               #se il valore di difference è nullo:
    if difference['country_code'].iloc[i] in diz_lenders:                                 
      difference['difference_amount'].iloc[i]=diz_lenders[difference['country_code'][i]]   #denaro prestato
    else:
      difference['difference_amount'].iloc[i]=diz_borrowers[difference['country_code'][i]]  #denaro preso a prestito

C:\Users\matti\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [68]:
difference

,country_code,difference_amount
0,AD,101.19
1,AE,101048.68
2,AF,-1961393.81
3,AI,29.69
4,AL,-4306159.66
...,...,...
215,YE,-3437537.68
216,YT,64.66
217,ZA,-975710.16
218,ZM,-1978309.01


In questo modo, abbiamo risolto il problema per tutti gli stati che avevano solo prestato o preso a prestito.

I valori positivi indicano gli stati che hanno prestato più di quanto hanno preso a prestito;
al contrario, i valori negativi indicano gli stati che hanno prestato meno di quanto hanno 
preso a prestito.


# 10- Which country has the highest ratio between the difference computed at the previous point and the population?


In [69]:
country=pd.read_csv(r'C:\Users\matti\OneDrive - Universita degli Studi di Milano-Bicocca\Uni\LM-DataScience\1-Anno\Computer Science\Final Project\Final Project CS\Kiva_Dataset\country_stats.csv')

In [70]:
country

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
0,India,IN,IND,Asia,Southern Asia,1339180127,21.90,0.62,68.32,11.70,6.30,5663.47,India
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.00,0.53,53.06,9.97,6.00,5442.90,Nigeria
2,Mexico,MX,MEX,Americas,Central America,129163276,46.20,0.76,76.97,13.30,8.55,16383.11,Mexico
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.50,0.55,66.36,8.11,5.09,5031.17,Pakistan
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.50,0.58,71.98,10.18,5.24,3341.49,Bangladesh
...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Somalia,SO,SOM,Africa,Eastern Africa,14742523,nan,nan,nan,nan,nan,nan,Somalia
170,Central African Republic,CF,CAF,Africa,Middle Africa,4659080,nan,0.35,51.46,7.10,4.23,587.47,Central African Republic
171,Samoa,WS,WSM,Oceania,Polynesia,196440,nan,0.70,nan,nan,nan,nan,Samoa
172,Palestine,PS,PS,Asia,Western Asia,4920724,nan,0.68,nan,nan,nan,nan,Palestine


In [71]:
ratio=pd.merge(country,difference,on='country_code')[['country_code','difference_amount','population','population_below_poverty_line']]

In [72]:
ratio.head()

,country_code,difference_amount,population,population_below_poverty_line
0,IN,-8862555.38,1339180127,21.90
1,NG,-7741916.28,190886311,70.00
2,MX,-25043776.72,129163276,46.20
3,PK,-24977887.42,197015955,29.50
4,BD,12860.30,164669751,31.50


In [73]:
ratio['ratio1']=ratio['difference_amount']/ratio['population']
ratio=ratio.sort_values(by='ratio1', ascending=False)
ratio[['country_code','ratio1']]

,country_code,ratio1
154,LU,0.23
156,NO,0.18
72,CA,0.16
146,AU,0.13
152,IS,0.11
...,...,...
116,MN,-4.99
82,SV,-6.54
108,AM,-7.83
93,PY,-7.92


Il Lussemburgo è il paese con il rapporto maggiore.

# 11- Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

In [74]:
country.head()

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
0,India,IN,IND,Asia,Southern Asia,1339180127,21.90,0.62,68.32,11.70,6.30,5663.47,India
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.00,0.53,53.06,9.97,6.00,5442.90,Nigeria
2,Mexico,MX,MEX,Americas,Central America,129163276,46.20,0.76,76.97,13.30,8.55,16383.11,Mexico
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.50,0.55,66.36,8.11,5.09,5031.17,Pakistan
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.50,0.58,71.98,10.18,5.24,3341.49,Bangladesh


In [75]:
country['population_not_poor']=country['population']-(country['population_below_poverty_line']*country['population']/100)

In [76]:
ratio['ratio2']=ratio['difference_amount']/country['population_not_poor']
ratio=ratio.sort_values(by='ratio2', ascending=False)
ratio[['country_code','ratio2']]

,country_code,ratio2
146,AU,2.71
132,AT,0.81
119,CH,0.22
72,CA,0.17
103,MY,0.15
...,...,...
158,PR,nan
162,SB,nan
166,XK,nan
165,PS,nan


L'Australia è il paese con il rapporto maggiore. 

# 12- For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

In [77]:
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,loans_duration,disburse_years
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.00,125.00,funded,General Store,Retail,...,3,2,1,NaN,female,true,irregular,field_partner,53 days 19:30:06,2013.00
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.00,400.00,funded,Used Clothing,Clothing,...,11,2,1,NaN,female,true,monthly,field_partner,96 days 14:25:07,2013.00
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.00,400.00,funded,General Store,Retail,...,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,37 days 13:10:05,2014.00
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.00,625.00,funded,Farming,Agriculture,...,21,2,1,user_favorite,female,true,monthly,field_partner,34 days 19:10:02,2014.00
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.00,425.00,funded,Farming,Agriculture,...,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,57 days 22:10:02,2013.00


Metodo poco efficace in quanto utilizza dei dataframe per memorizzare le informazioni:

In [78]:
loans_short=loans[0:1000]

In [79]:
loans_short['disburse_time']=pd.to_datetime(loans_short['disburse_time'])
loans_short['planned_expiration_time']=pd.to_datetime(loans_short['planned_expiration_time'])

C:\Users\matti\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\matti\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [80]:
dataframe=pd.DataFrame({'loan_id':[], 'anno':[], 'cash':[]})

#Inizializzo il contatore per le righe del nuovo dataset
k=0

for i in range(0,len(loans_short)):
  
  loan_id=loans_short.iloc[i]['loan_id']
  amount=loans_short.iloc[i]['loan_amount']
  anno_inizio=loans_short.iloc[i]['disburse_time'].year
  anno_fine=loans_short.iloc[i]['planned_expiration_time'].year
  giorni=(loans_short.iloc[i]['planned_expiration_time']-loans_short.iloc[i]['disburse_time']).days
  
  # Non ha senso considerare una planned expiration time precedente alla disburse time
  if giorni > 1:
    if anno_inizio==anno_fine:
      dataframe.loc[k]=[loan_id,anno_inizio,amount]
      k+=1

    #Eseguo il calcolo per i loans aperti e chiusi in anni differenti
    else:
      #Creo una lista con tutti gli anni.
      anni=list(range(anno_inizio,anno_fine+1))  
    
      righe=len(anni)
      indice_righe=list(range(k,k+righe+1))

      #Primo anno
      anno=anni[0]
      n=365-pd.to_datetime(loans_short['disburse_time'][i]).dayofyear
      cash=(n*amount)/giorni
      dataframe.loc[indice_righe[0]]=[loan_id,anno,cash]

      #Anni centrali
      for l in range(1,len(anni)-1):
        anno=anni[l]
        cash=(365*amount)/giorni
        dataframe.loc[indice_righe[l]]=[loan_id,anno,cash]

      #Ultimo anno
      anno=anni[-1]
      n=pd.to_datetime(loans_short['planned_expiration_time'][i]).dayofyear
      cash=(n*amount)/giorni
      dataframe.loc[indice_righe[-1]]=[loan_id,anno,cash]

      k=k+righe

In [81]:
dataframe

,loan_id,anno,cash
0,657307.00,2013.00,21.23
2,657259.00,2013.00,45.83
4,658010.00,2014.00,400.00
5,659347.00,2014.00,625.00
6,656933.00,2013.00,104.39
...,...,...,...
1010,659317.00,2013.00,22.17
1012,660410.00,2014.00,125.00
1013,663077.00,2014.00,225.00
1014,663849.00,2014.00,1200.00


In [82]:
new_dataframe=dataframe.groupby('anno')['cash'].sum()

In [83]:
new_dataframe

anno
2012.00   252775.00
2013.00    10901.81
2014.00   211475.92
2015.00   108225.00
2016.00   125124.90
2017.00    21025.00
Name: cash, dtype: float64

Metodo più efficace (con l'utilizzo di liste):

In [84]:
loans['disburse_time']=pd.to_datetime(loans['disburse_time'])
loans['planned_expiration_time']=pd.to_datetime(loans['planned_expiration_time'])

In [85]:
dataframe=pd.DataFrame({'anno':[], 'cash':[]})
lista_anni=[]
lista_amount=[]

amounts=[x for x in loans['loan_amount']]
anni_inizio=[x.year for x in loans['disburse_time']]
anni_fine=[x.year for x in loans['planned_expiration_time']]

for i in range(0,len(loans)):
  
  amount=amounts[i]
  anno_inizio=anni_inizio[i]
  anno_fine=anni_fine[i]
  giorni=(loans.iloc[i]['planned_expiration_time']-loans.iloc[i]['disburse_time']).days

  #Non ha senso considerare una planned expiration time precedente alla disburse time.
  #Non sono quindi considerati tali loans.
  if giorni > 1:
    if anno_inizio==anno_fine:
      lista_anni.append(anno_inizio)
      lista_amount.append(amount)
      #Inserisco direttamente l'amount in quanto il n giorni dell'anno equivale
      #al numero di giorni totale.

    #Eseguo il calcolo per i loans aperti e chiusi in anni differenti
    else:
      #Creo una lista con tutti gli anni.
      anni=list(range(anno_inizio,anno_fine+1))  
    
      #Primo anno
      anno=anni[0]
      n=365-pd.to_datetime(loans['disburse_time'][i]).dayofyear
      cash=(n*amount)/giorni
      lista_anni.append(anno)
      lista_amount.append(cash)

      #Anni centrali
      for l in range(1,len(anni)-1):
        anno=anni[l]
        cash=(365*amount)/giorni
        lista_anni.append(anno)
        lista_amount.append(cash)

      #Ultimo anno
      anno=anni[-1]
      n=pd.to_datetime(loans['planned_expiration_time'][i]).dayofyear
      cash=(n*amount)/giorni
      lista_anni.append(anno)
      lista_amount.append(cash)

In [86]:
dataframe=pd.DataFrame({'anno':lista_anni, 'cash':lista_amount})

In [87]:
dataframe

,anno,cash
0,2013,21.23
1,2014,106.13
2,2013,45.83
3,2014,354.17
4,2014,400.00
...,...,...
1179868,2016,142.65
1179869,2015,92.06
1179870,2016,7.94
1179871,2015,166.39


In [88]:
new_dataframe=pd.DataFrame(dataframe.groupby('anno')['cash'].sum())

In [89]:
new_dataframe

,cash
anno,
2011,631678.34
2012,112692261.55
2013,123415680.29
2014,150210963.41
2015,154609678.39
2016,153006054.15
2017,164556145.63
2018,5097248.35


In [90]:
sum(new_dataframe['cash'])

864219710.1047662